## OVERSAMPLING + DOWNSAMPLING in DomainData.over_sampling

In [1]:
import numpy as np
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend
import pandas as pd
import tensorflow as tf
import random
import os
from keras import layers
from sklearn.metrics import confusion_matrix

NUM_TOKEN = 5000
MAX_PRO_LEN = 64
MAX_TXT_LEN = 256
NO_EPO = 4
NO_BAT = 128

MACHINE_1_P = "./data/set1_machine.json"
HUMAN_1_P = "./data/set1_human.json"
MACHINE_2_P = "./data/set2_machine.json"
HUMAN_2_P = "./data/set2_human.json"
TEST_P = "./data/test.json"
RANDOM_SEED = 42
MACHINE_IND = 0
HUMAN_IND = 1
TEST_FRA = 0.2

class DomainData:
    """
    train_test_split, pad_sequence, PCA, class_weight
    """
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def t_t_spli(self, test_size, random_state):
        self.random_state = random_state
        self.train_x, self.test_x, self.train_y, self.test_y = train_test_split(self.x, self.y, test_size=test_size, stratify = self.y, random_state = random_state)
        self.train_x = self.train_x.reset_index(drop=True)
        self.train_y = self.train_y.reset_index(drop=True)
        self.test_x = self.test_x.reset_index(drop=True)
        self.test_y = self.test_y.reset_index(drop=True)
        

    def add_padding(self, padding, prompt_len, txt_len):
        self.train_prompt = self.train_x["prompt"]
        self.train_txt = self.train_x["txt"]
        self.train_label = self.train_y.to_numpy()
        self.test_prompt = self.test_x["prompt"]
        self.test_txt = self.test_x["txt"]
        self.test_label = self.test_y.to_numpy()
        unique_classes = np.unique(self.train_label)
        class_weights = class_weight.compute_class_weight("balanced", classes=unique_classes, y=self.train_y)
        self.class_weights = dict(zip(unique_classes, class_weights))
        
        self.prompt_len = prompt_len
        self.txt_len = txt_len
        
        self.train_prompt = pad_sequences(self.train_prompt, padding=padding, maxlen=prompt_len)
        self.train_txt = pad_sequences(self.train_txt, padding=padding, maxlen=txt_len)
        self.test_prompt = pad_sequences(self.test_prompt, padding=padding, maxlen=prompt_len)
        self.test_txt = pad_sequences(self.test_txt, padding=padding, maxlen=txt_len)
        
        
    def down_sampling(self):
        mac_ind = self.train_y[self.train_y == MACHINE_IND].index.to_list()
        hum_ind = self.train_y[self.train_y == HUMAN_IND].index.to_list()
        lower = min(len(mac_ind), len(hum_ind))
        sel_lit = mac_ind[:lower] + hum_ind[:lower]
        self.train_x = self.train_x.iloc[sel_lit]
        self.train_y = self.train_y.iloc[sel_lit]
        random.shuffle(sel_lit)

    def over_sampling(self, upper_fra):
        ## ______________________ oversampling + downsampling ______________________
        mac_ind = self.train_y[self.train_y == MACHINE_IND].index.to_list()
        hum_ind = self.train_y[self.train_y == HUMAN_IND].index.to_list()
        lower = min(len(mac_ind), len(hum_ind))
        if lower == len(mac_ind):
            upper = int(lower*upper_fra) if lower*upper_fra < len(hum_ind) else len(hum_ind)
            major = hum_ind[:upper]
            minor = mac_ind[:lower]
    
        else:
            upper = int(lower*upper_fra) if lower*upper_fra < len(mac_ind) else len(mac_ind)
            major = mac_ind[:upper]
            minor = hum_ind[:lower]
        
        add_n = upper - lower
        oversampled = []
        while(len(oversampled) < add_n):
            oversampled.append(random.choice(mac_ind))
        sel_lit = major + minor + oversampled
        random.shuffle(sel_lit)
        
        self.train_x = self.train_x.iloc[sel_lit]
        self.train_y = self.train_y.iloc[sel_lit]
    
    def test_down(self, frac = 1):
        mac_ind = self.test_y[self.test_y == MACHINE_IND].index.to_list()
        hum_ind = self.test_y[self.test_y == HUMAN_IND].index.to_list()
        lower = min(len(mac_ind), len(hum_ind))
        if frac > 1:
            sel_lit = mac_ind[:lower] + hum_ind[:int(lower/frac)]
        else:
            sel_lit = mac_ind[:int(lower*frac)] + hum_ind[:lower]
        self.test_x = self.test_x.iloc[sel_lit]
        self.test_y = self.test_y.iloc[sel_lit]
        random.shuffle(sel_lit)




def f1_loss(y_true, y_pred):
    # Calculate precision and recall
    tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)))
    fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)))
    precision = tp / (tp + fp + backend.epsilon())
    recall = tp / (tp + fn + backend.epsilon())
    
    # Calculate F1 score
    f1_score = 2 * precision * recall / (precision + recall + backend.epsilon())
    
    # Return negative F1 score as the loss (to minimize it)
    return -f1_score

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience = 3)
random.seed(RANDOM_SEED)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if len(tf.config.list_physical_devices('GPU')):
    tf.config.list_physical_devices('GPU')
    print("Using GPU")


2023-04-19 14:37:12.300227: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-19 14:37:12.376425: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 14:37:13.379043: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1
Using GPU


2023-04-19 14:37:15.184966: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 14:37:15.219455: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 14:37:15.219996: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [6]:
# import matplotlib.pyplot as plt
# ## _______________ Plot distribution _______________
# man_1_df = pd.read_json(HUMAN_1_P)
# man_1_df["label"] = HUMAN_IND
# mac_1_df = pd.read_json(MACHINE_1_P).drop("machine_id", axis = 1)
# mac_1_df["label"] = MACHINE_IND
# domain_1_df = pd.concat([man_1_df, mac_1_df])

# domain_1_df["length_p"] = domain_1_df["prompt"].apply(lambda x: len(x))
# domain_1_df["length_t"] = domain_1_df["txt"].apply(lambda x: len(x))

# domain_1_df["length_t"].plot.hist()
# plt.xlim(100, 1000)
# plt.show()

# print(domain_1_df.head())

In [2]:
## _______________ Read data from domain 1 _______________
man_1_df = pd.read_json(HUMAN_1_P)
man_1_df["label"] = HUMAN_IND
mac_1_df = pd.read_json(MACHINE_1_P).drop("machine_id", axis = 1)
mac_1_df["label"] = MACHINE_IND
domain_1_df = pd.concat([man_1_df, mac_1_df])

domain_1 = DomainData(domain_1_df[["prompt", "txt"]], domain_1_df["label"])
domain_1.t_t_spli(TEST_FRA, RANDOM_SEED)
domain_1.over_sampling(1.6)
domain_1.test_down()
domain_1.add_padding('post', MAX_PRO_LEN, MAX_TXT_LEN)


In [4]:
# best: 633  /642: epo->4. bat->128
epo_size = 4
batch_size = 128

## _______________ RNN on Prompts _______________
p_model_rnn= tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=NUM_TOKEN,
        output_dim=128,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5)
    # tf.keras.layers.LSTM(32)
])

## _______________ RNN on txt _______________
t_model_rnn= tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=NUM_TOKEN,
        output_dim=128,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    # tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5)
    # tf.keras.layers.LSTM(64)
])

## _______________ Combined _______________
prompt_input = layers.Input(shape=(MAX_PRO_LEN,))
text_input = layers.Input(shape=(MAX_TXT_LEN,))
prompt_output = p_model_rnn(prompt_input)
text_output = t_model_rnn(text_input)
merged = layers.concatenate([prompt_output, text_output])
merged = layers.Dense(64, activation='tanh')(merged)
merged = layers.Dropout(0.5)(merged)
merged = layers.Dense(1, activation='sigmoid')(merged)
combined_model = tf.keras.models.Model(inputs=[prompt_input, text_input], outputs=merged)
combined_model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy', f1_loss])

# combined_model.fit([domain_1.train_prompt, domain_1.train_txt], domain_1.train_label, epochs=epo_size, batch_size=batch_size, validation_split=0.2, class_weight=domain_1.class_weights, callbacks = [callback])
combined_model.fit([domain_1.train_prompt, domain_1.train_txt], domain_1.train_label, epochs=epo_size, batch_size=batch_size, validation_split=0.2, callbacks = [callback])
combined_model.save("combined_model.h5")

## evaluate
loss, accuracy, f1 = combined_model.evaluate([domain_1.test_prompt, domain_1.test_txt], domain_1.test_label, verbose=False)
print("loss: ", loss)
print("accuracy", accuracy)
comb_1_pre_rnn = combined_model.predict([domain_1.test_prompt, domain_1.test_txt])
comb_1_pre_rnn = np.round(comb_1_pre_rnn).flatten()
confusion = confusion_matrix(domain_1.test_label, comb_1_pre_rnn)
print(confusion)
f1 = f1_score(domain_1.test_label, comb_1_pre_rnn)
print("f1-score: ", f1)


Epoch 1/4


2023-04-19 14:38:39.703390: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-04-19 14:38:39.932744: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-04-19 14:38:42.074962: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ign

56/56 [==============================] - 21s 248ms/step - loss: 0.5900 - accuracy: 0.6882 - f1_loss: -0.6805 - val_loss: 0.3712 - val_accuracy: 0.8276 - val_f1_loss: -0.8302
Epoch 2/4
56/56 [==============================] - 10s 181ms/step - loss: 0.2089 - accuracy: 0.9192 - f1_loss: -0.9195 - val_loss: 0.2019 - val_accuracy: 0.9202 - val_f1_loss: -0.9158
Epoch 3/4
56/56 [==============================] - 8s 145ms/step - loss: 0.0587 - accuracy: 0.9798 - f1_loss: -0.9797 - val_loss: 0.2094 - val_accuracy: 0.9230 - val_f1_loss: -0.9161
Epoch 4/4
56/56 [==============================] - 7s 117ms/step - loss: 0.0155 - accuracy: 0.9962 - f1_loss: -0.9963 - val_loss: 0.2650 - val_accuracy: 0.9230 - val_f1_loss: -0.9215
loss:  0.41972485184669495
accuracy 0.8785714507102966
44/44 [==============================] - 3s 23ms/step
[[579 121]
 [ 49 651]]
f1-score:  0.8845108695652174


### Domain 2: Transfer Learning

In [ ]:
## _______________ Read data from domain 2 _______________
man_2_df = pd.read_json(HUMAN_2_P)
man_2_df["label"] = HUMAN_IND
mac_2_df = pd.read_json(MACHINE_2_P).drop("machine_id", axis = 1)
mac_2_df["label"] = MACHINE_IND
domain_2_df = pd.concat([man_2_df, mac_2_df])

domain_2 = DomainData(domain_2_df[["prompt", "txt"]], domain_2_df["label"])
domain_2.t_t_spli(TEST_FRA, RANDOM_SEED)
domain_2.add_padding('post', MAX_PRO_LEN, MAX_TXT_LEN)

In [ ]:
epo_size = NO_EPO
batch_size = 40


prompt_input = layers.Input(shape=(MAX_PRO_LEN,))
text_input = layers.Input(shape=(MAX_TXT_LEN,))

## _______________ Load model and continue training _______________
combined_model_2 = tf.keras.models.load_model("combined_model.h5", custom_objects={ 'f1_loss': f1_loss })
combined_model_2.layers.pop()
new_layer = layers.Dense(1, activation='sigmoid')(combined_model_2.output)
combined_model_2 = tf.keras.models.Model(inputs=combined_model_2.input, outputs=new_layer)
combined_model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_loss])

combined_model_2.fit([domain_2.train_prompt, domain_2.train_txt], domain_2.train_label, epochs=epo_size, batch_size=batch_size, validation_split=0.2, class_weight=domain_2.class_weights, callbacks = [callback])
combined_model_2.save("combined_model_2.h5")

## evaluate
loss, accuracy, f1 = combined_model_2.evaluate([domain_2.test_prompt, domain_2.test_txt], domain_2.test_label, verbose=False)
print("loss: ", loss)
print("accuracy", accuracy)
comb_2_pre_rnn = combined_model_2.predict([domain_2.test_prompt, domain_2.test_txt])
comb_2_pre_rnn = np.round(comb_2_pre_rnn).flatten()
confusion = confusion_matrix(domain_2.test_label, comb_2_pre_rnn)
print(confusion)
f1 = f1_score(domain_2.test_label, comb_2_pre_rnn)
print("f1-score: ", f1)

Epoch 1/6
1/1 [==============================] - ETA: 0s - loss: 0.6710 - accuracy: 0.8150 - f1_loss: 0.0000e+00

### Domain 2: Weighted Domain Data

In [ ]:
over_fra = 1.6
weight_fra = 300

## _______________ Read data from domain 1 _______________
man_1_df = pd.read_json(HUMAN_1_P)
man_1_df["label"] = HUMAN_IND
mac_1_df = pd.read_json(MACHINE_1_P).drop("machine_id", axis = 1)
mac_1_df["label"] = MACHINE_IND
domain_1_df = pd.concat([man_1_df, mac_1_df])

domain_1 = DomainData(domain_1_df[["prompt", "txt"]], domain_1_df["label"])
domain_1.t_t_spli(TEST_FRA, RANDOM_SEED)
# domain_1.down_sampling()
domain_1.add_padding('post', MAX_PRO_LEN, MAX_TXT_LEN)

## _______________ Read data from domain 2 _______________
man_2_df = pd.read_json(HUMAN_2_P)
man_2_df["label"] = HUMAN_IND
mac_2_df = pd.read_json(MACHINE_2_P).drop("machine_id", axis = 1)
mac_2_df["label"] = MACHINE_IND
domain_2_df = pd.concat([man_2_df, mac_2_df])

domain_2 = DomainData(domain_2_df[["prompt", "txt"]], domain_2_df["label"])
domain_2.t_t_spli(TEST_FRA, RANDOM_SEED)
# domain_2.over_sampling(over_fra)
domain_2.test_down()
domain_2.add_padding('post', MAX_PRO_LEN, MAX_TXT_LEN)



## _______________ weight data _______________
sample_weight_1 = np.ones(len(domain_1.train_label))
sample_weight_2 = np.ones(len(domain_2.train_label))
sample_weight_2 *= weight_fra
sample_weight = np.concatenate([sample_weight_1, sample_weight_2])

train_prompt = np.concatenate([domain_1.train_prompt, domain_2.train_prompt])
train_txt = np.concatenate([domain_1.train_txt, domain_2.train_txt])
train_label = np.concatenate([domain_1.train_label, domain_2.train_label])

data = list(zip(train_prompt, train_txt, train_label, sample_weight))
random.shuffle(data)

train_prompt, train_txt, train_label, sample_weight = zip(*data)
train_prompt = np.array(train_prompt)
train_txt = np.array(train_txt)
train_label = np.array(train_label)
sample_weight = np.array(sample_weight)

In [ ]:
## _______________ RNN on Prompts _______________
p_model_rnn= tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=NUM_TOKEN,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(64, activation='relu')
])

## _______________ RNN on txt _______________
t_model_rnn= tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=NUM_TOKEN,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(64, activation='relu')
])

## _______________ Combined _______________
prompt_input = layers.Input(shape=(MAX_PRO_LEN,))
text_input = layers.Input(shape=(MAX_TXT_LEN,))
prompt_output = p_model_rnn(prompt_input)
text_output = t_model_rnn(text_input)
merged = layers.concatenate([prompt_output, text_output])
merged = layers.Dense(64, activation='relu')(merged)
merged = layers.Dropout(0.5)(merged)
merged = layers.Dense(1, activation='sigmoid')(merged)
combined_model = tf.keras.models.Model(inputs=[prompt_input, text_input], outputs=merged)
combined_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_loss])

combined_model.fit([train_prompt, train_txt], train_label, epochs=NO_EPO, batch_size=NO_BAT, validation_split=0.2, class_weight=domain_2.class_weights, sample_weight = sample_weight, callbacks = [callback])
combined_model.save("combined_model_weight.h5")

## evaluate
loss, accuracy, f1 = combined_model.evaluate([domain_2.test_prompt, domain_2.test_txt], domain_2.test_label, verbose=False)
print("loss: ", loss)
print("accuracy", accuracy)
comb_2_pre_rnn = combined_model.predict([domain_2.test_prompt, domain_2.test_txt])
comb_2_pre_rnn = np.round(comb_2_pre_rnn).flatten()
confusion = confusion_matrix(domain_2.test_label, comb_2_pre_rnn)
print(confusion)
f1 = f1_score(domain_2.test_label, comb_2_pre_rnn)
print("f1-score: ", f1)


2023-04-19 12:55:00.200953: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 12:55:00.201510: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 12:55:00.202118: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 12:55:01.198719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 12:55:01.199246: I tensorflow/compile

Epoch 1/4


2023-04-19 12:55:14.966763: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-04-19 12:55:15.319235: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-04-19 12:55:18.468233: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ign

396/396 [==============================] - 66s 137ms/step - loss: 1.1827 - accuracy: 0.9685 - f1_loss: -0.9837 - val_loss: 1.3052 - val_accuracy: 0.9715 - val_f1_loss: -0.9855
Epoch 2/4
396/396 [==============================] - 32s 82ms/step - loss: 0.7947 - accuracy: 0.9712 - f1_loss: -0.9853 - val_loss: 0.9133 - val_accuracy: 0.9721 - val_f1_loss: -0.9858
Epoch 3/4
396/396 [==============================] - 30s 76ms/step - loss: 0.1421 - accuracy: 0.9731 - f1_loss: -0.9863 - val_loss: 0.7067 - val_accuracy: 0.9745 - val_f1_loss: -0.9870
Epoch 4/4
396/396 [==============================] - 30s 77ms/step - loss: 0.2143 - accuracy: 0.9771 - f1_loss: -0.9883 - val_loss: 0.6982 - val_accuracy: 0.9731 - val_f1_loss: -0.9862
loss:  0.2955428957939148
accuracy 0.8899999856948853
4/4 [==============================] - 3s 49ms/step
[[49  1]
 [ 9 41]]
f1-score:  0.8913043478260869


### test on the test set

In [ ]:
DOMAIN_SPL = 600

test_df = pd.read_json(TEST_P)
test_prompt = pad_sequences(test_df["prompt"], padding="post", maxlen=MAX_PRO_LEN)
test_txt = pad_sequences(test_df["txt"], padding="post", maxlen=MAX_TXT_LEN)

print(test_prompt)

model_1 = tf.keras.models.load_model("combined_model.h5", custom_objects={ 'f1_loss': f1_loss })
model_2 = tf.keras.models.load_model("combined_model_weight.h5", custom_objects={ 'f1_loss': f1_loss })

pred = []
pred += model_1.predict([test_prompt[:DOMAIN_SPL], test_txt[:DOMAIN_SPL]]).tolist()
pred += model_2.predict([test_prompt[DOMAIN_SPL:], test_txt[DOMAIN_SPL:]]).tolist()
pred = [int(i) for i in np.round(pred).flatten()]


[[2109 2263   86 ...    0    0    0]
 [2267 1890 3460 ...    0    0    0]
 [1602 2770 4021 ...    0    0    0]
 ...
 [  43 1661   46 ...   49   49   17]
 [1737   36 2607 ...    0    0    0]
 [  44   40   47 ... 3316 4183   17]]


2023-04-19 14:03:08.892995: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 14:03:08.893706: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 14:03:08.894128: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 14:03:10.119621: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 14:03:10.120453: I tensorflow/compile

 5/19 [======>.......................] - ETA: 0s

2023-04-19 14:03:23.395478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


13/13 [==============================] - 3s 40ms/step


In [ ]:
pred_df = pd.DataFrame(pred)
pred_df.columns = ["Predicted"]
pred_df.index.names = ['Id']

pred_df.to_csv("./data/result2.csv")

### Load and evaluate

In [ ]:
combined_model_1 = tf.keras.models.load_model("combined_model.h5", custom_objects={ 'f1_loss': f1_loss })
## evaluate
loss, accuracy, f1 = combined_model_1.evaluate([domain_1.test_prompt, domain_1.test_txt], domain_1.test_label, verbose=False)
print("loss: ", loss)
print("f1-score: ", f1)
print("accuracy", accuracy)
comb_1_pre_rnn = combined_model_1.predict([domain_1.test_prompt, domain_1.test_txt])
comb_1_pre_rnn = np.round(comb_1_pre_rnn).flatten()
confusion = confusion_matrix(domain_1.test_label, comb_1_pre_rnn)
print(confusion)

2023-04-19 12:59:15.978710: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 12:59:15.979581: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 12:59:15.980598: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 12:59:17.032635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-19 12:59:17.033154: I tensorflow/compile

loss:  0.5814876556396484
f1-score:  -0.46047720313072205
accuracy 0.8568571209907532
110/110 [==============================] - 5s 21ms/step
[[1553  197]
 [ 304 1446]]


In [ ]:
combined_model_2 = tf.keras.models.load_model("combined_model_weight.h5", custom_objects={ 'f1_loss': f1_loss })
## evaluate
loss, accuracy, f1 = combined_model_2.evaluate([domain_2.test_prompt, domain_2.test_txt], domain_2.test_label, verbose=False)
print("loss: ", loss)
print("accuracy", accuracy)
comb_2_pre_rnn = combined_model_2.predict([domain_2.test_prompt, domain_2.test_txt])
comb_2_pre_rnn = np.round(comb_2_pre_rnn).flatten()
confusion = confusion_matrix(domain_2.test_label, comb_2_pre_rnn)
print(confusion)
f1 = f1_score(domain_2.test_label, comb_2_pre_rnn)
print("f1-score: ", f1)

loss:  0.31285056471824646
accuracy 0.8899999856948853
4/4 [==============================] - 3s 46ms/step
[[49  1]
 [ 9 41]]
f1-score:  0.8913043478260869
